In [1]:
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)

In [2]:
import sys
sys.path.append('/home/kit/kit/donson_intern/langchain/Dungeons&dragons')
from DialogueAgent import DialogueAgent
from DialogueSimulator import DialogueSimulator

sys.path.append('/home/kit/kit/donson_intern/langchain')
from ChatGLM3 import ChatGLM3
model_path = "/data/tds/ChatGLM3/chatglm3-6b"
llm = ChatGLM3()
llm.load_model(model_name_or_path=model_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
llm("你好")

'Hello! How can I assist you today?'

In [4]:
character_names = ["Harry Potter", "Ron Weasley", "Hermione Granger", "Argus Filch"]
storyteller_name = "Dungeon Master"
quest = "Find all of Lord Voldemort's seven horcruxes."
word_limit = 50 # word limit for task brainstorming

In [5]:
game_description = f"""Here is the topic for a Dungeons & Dragons game: {quest}.
The characters are: {*character_names,}.
The story is narrated by the storyteller, {storyteller_name}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of a Dungeons & Dragons player.")
game_description

"Here is the topic for a Dungeons & Dragons game: Find all of Lord Voldemort's seven horcruxes..\nThe characters are: ('Harry Potter', 'Ron Weasley', 'Hermione Granger', 'Argus Filch').\nThe story is narrated by the storyteller, Dungeon Master."

In [6]:
def generate_character_description(character_name):
    player_descriptor_human_message = HumanMessage(content=
            f"""{game_description}
 Please reply with a creative description of the character, {character_name}, in {word_limit} words or less. 
 Speak directly to {character_name}.
 Do not add anything else."""
            )
    character_specifier_prompt = [
        player_descriptor_system_message,
        player_descriptor_human_message
    ]
    prompt = player_descriptor_system_message.content + '\n' + player_descriptor_human_message.content
    print(prompt)
    character_description = llm(prompt)
 
    return character_description
 
 
 
def generate_character_system_message(character_name, character_description):
    return SystemMessage(content=(
    f"""{game_description}
Your name is {character_name}. 
Your character description is as follows: {character_description}.
You will propose actions you plan to take and {storyteller_name} will explain what happens when you take those actions.
Speak in the first person from the perspective of {character_name}.
For describing your own body movements, wrap your description in '\*'.
Do not change roles!
Do not speak from the perspective of anyone else.
Remember you are {character_name}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else.
"""
    ))
 
 
#  为每个角色生成描述和系统信息
character_descriptions = [generate_character_description(character_name) for character_name in character_names]
character_system_messages = [generate_character_system_message(character_name, character_description) for character_name, character_description in zip(character_names, character_descriptions)]
 
 
storyteller_specifier_prompt_human_massage = HumanMessage(content=
        f"""{game_description}
Please reply with a creative description of the storyteller, {storyteller_name}, in {word_limit} words or less. 
Speak directly to {storyteller_name}.
Do not add anything else."""
        )
storyteller_specifier_prompt = [
    player_descriptor_system_message,
    storyteller_specifier_prompt_human_massage
]
prompt = player_descriptor_system_message.content + '\n' + storyteller_specifier_prompt_human_massage.content
print(prompt)
storyteller_description = llm(prompt)
 

storyteller_system_message = SystemMessage(content=(
f"""{game_description}
You are the storyteller, {storyteller_name}. 
Your description is as follows: {storyteller_description}.
The other players will propose actions to take and you will explain what happens when they take those actions.
Speak in the first person from the perspective of {storyteller_name}.
Do not change roles!
Do not speak from the perspective of anyone else.
Remember you are the storyteller, {storyteller_name}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else.
"""
))

You can add detail to the description of a Dungeons & Dragons player.
Here is the topic for a Dungeons & Dragons game: Find all of Lord Voldemort's seven horcruxes..
The characters are: ('Harry Potter', 'Ron Weasley', 'Hermione Granger', 'Argus Filch').
The story is narrated by the storyteller, Dungeon Master.
 Please reply with a creative description of the character, Harry Potter, in 50 words or less. 
 Speak directly to Harry Potter.
 Do not add anything else.
You can add detail to the description of a Dungeons & Dragons player.
Here is the topic for a Dungeons & Dragons game: Find all of Lord Voldemort's seven horcruxes..
The characters are: ('Harry Potter', 'Ron Weasley', 'Hermione Granger', 'Argus Filch').
The story is narrated by the storyteller, Dungeon Master.
 Please reply with a creative description of the character, Ron Weasley, in 50 words or less. 
 Speak directly to Ron Weasley.
 Do not add anything else.
You can add detail to the description of a Dungeons & Dragons play

In [7]:
print('Storyteller Description:') 
print(storyteller_description) 
for character_name, character_description in zip(character_names, character_descriptions): 
    print(f'{character_name} Description:') 
    print(character_description)

Storyteller Description:
Creatures lurk in the shadows, waiting for their moment to strike. Harry Potter, Ron Weasley, Hermione Granger, and Argus Filch must band together to find Lord Voldemort's seven horcruxes, before they fall into the wrong hands. The fate of the wizarding world rests in their hands.
Harry Potter Description:
"Harry Potter is a young wizard with a rare gift, he is the only one who can survive in the magical world without the help of magic. He is known for his bravery and intelligence, and is a true hero among his friends. He is on a quest to find all of Lord Voldemort's seven horcruxes and put an end to his evil plans once and for all."
Ron Weasley Description:
Ron Weasley is a wizard with a mischievous grin and a love of practical jokes. He is an expert Quidditch player and a skilled tracker, able to follow his prey through the forest. He is also a loyal friend and a brave defender of the innocent.
Hermione Granger Description:
"Hermione Granger is a brilliant an

In [8]:
quest_specifier_prompt_system_message = SystemMessage(content="You can make a task more specific.")
quest_specifier_prompt_human_message = HumanMessage(content=
        f"""{game_description}
You are the storyteller, {storyteller_name}.
Please make the quest more specific. Be creative and imaginative.
Please reply with the specified quest in {word_limit} words or less. 
Speak directly to the characters: {*character_names,}.
Do not add anything else."""
        )
quest_specifier_prompt = [
    quest_specifier_prompt_system_message,
    quest_specifier_prompt_human_message
]

prompt = quest_specifier_prompt_system_message.content + '\n' + quest_specifier_prompt_human_message.content
specified_quest = llm(prompt)
print(f"Original quest:{quest}")
print(f"Detailed quest:{specified_quest}")

Original quest:Find all of Lord Voldemort's seven horcruxes.
Detailed quest:Lord Voldemort's seven horcruxes are scattered throughout the wizarding world. Your quest is to find and destroy each of them, ensuring the defeat of the dark lord himself. You will be aided by your friends Hermione and Ron, as well as Argus Filch, who is in league with the Ministry of Magic. Good luck on your quest.


In [9]:
characters = []
for character_name, character_system_message in zip(character_names, character_system_messages):
    characters.append(DialogueAgent(
        name=character_name,
        system_message=character_system_message, 
        model=llm))
storyteller = DialogueAgent(name=storyteller_name,
                     system_message=storyteller_system_message, 
                     model=llm)

In [10]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    if step % 2 == 0:
        idx = 0
    else:
        idx = (step//2) % (len(agents)-1) + 1
    return idx

In [11]:
max_iters = 10
n = 0
simulator = DialogueSimulator(
    agents=[storyteller] + characters,
    selection_function=select_next_speaker
)
simulator.reset()
simulator.inject(storyteller_name, specified_quest)
print(f"({storyteller_name}): {specified_quest}")
print('')

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('')
    n += 1

(Dungeon Master): Lord Voldemort's seven horcruxes are scattered throughout the wizarding world. Your quest is to find and destroy each of them, ensuring the defeat of the dark lord himself. You will be aided by your friends Hermione and Ron, as well as Argus Filch, who is in league with the Ministry of Magic. Good luck on your quest.

(Harry Potter): *I look at the map of the wizarding world, studying the locations of Lord Voldemort's seven horcruxes.* "The first one is located in the Forbidden Forest, near the location of my parents' deaths. I must be careful, as the forest is filled with dangerous creatures and magic."

(Dungeon Master): *Harry looks at the map and realizes the first horcrux is in the Forbidden Forest. He knows it's a dangerous place, but he must face the challenge and find the horcrux.

Harry Potter: *I make my way to the Forbidden Forest, keeping a watchful eye out for any signs of danger.*

(Ron Weasley): *I follow Harry into the forest, keeping a close eye on my